In [3]:
import os

import numpy as np
import pandas as pd
from astropy.io import fits
import importlib

import statmorph
from statmorph.utils.image_diagnostics import make_figure
from utils.data import segmentation_map, circular_mask, zoom_in, create_morph_df
from utils.plots import plot_light, display_img
%matplotlib inline

In [4]:
data_dir = 'cluster_data/'
drz_file = os.path.join(data_dir, 'hlsp_clash_hst_acs-wfc3ir_a383_total_v1_drz.fits')
whz_file = os.path.join(data_dir, 'hlsp_clash_hst_acs-wfc3ir_a383_total_v1_whz.fits')
hdulist = fits.open(drz_file)
drz = hdulist[0].data

In [5]:
full_segmap = segmentation_map(drz, nsigma=10.)

In [ ]:
full_source_morphs = statmorph.source_morphology(image=drz,
                                                 segmap=full_segmap,
                                                 gain=2.25)

In [20]:
def func(src):
    return src.r50      # sort on half-light radius to get BCG

full_source_morphs.sort(reverse=True, key=func)
bcg = full_source_morphs[1]
full_morph_df = create_morph_df(full_source_morphs)

In [ ]:
make_figure(bcg)

In [ ]:
zoomed_region, zoom_size = zoom_in(bcg, drz)
plot_light(bcg, zoomed_region, zoom_size)

In [ ]:
center = (bcg.xc_centroid, bcg.yc_centroid)
rads = bcg.r50*np.linspace(1, 5, num=20)
morphs_list = []
for radius in rads:
    mask = circular_mask(drz, center, radius)
    # display_img(image=drz, mask=mask)
    sm_mask = mask.astype(int)
    morph = statmorph.source_morphology(drz,
                                     segmap=sm_mask,
                                     gain=2.25)
    morphs_list.append(morph[0])

In [30]:
morph_df = create_morph_df(morphs_list)

In [ ]:
morph_df